In [ ]:
!pip install qdrant_client
!pip install datasets
!pip install tdqm
!pip install pymilvus
!pip install faiss
!pip install accelerate
!pip install transformers
!pip install -U bitsandbytes
!pip install datasets
!pip install rouge-score
!pip install pymorphy3
!pip install peft
!pip install flash_attn
!pip install langchain
!pip install langchain_community
!pip install pymorphy2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.7/306.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_mach

In [ ]:

!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 63.8 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=fe44c24c62773c453c839d6d2dcaf9e30a8ee1343de0f09fe128ca8eb5a1c03a
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt


Faiss base

In [ ]:
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
import faiss
# from transformers import AutoTokenizer

model_name = ""
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# def vectorize_batch(batch):
#     max_words = 25
#     Y, X = list(zip(*batch))
#     X = [vocab(tokenizer(text)) for text in X]
#     X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X] ## Bringing all samples to max_words length.

#     return torch.tensor(X, dtype=torch.int32), torch.tensor(Y) - 1

news_dataset = load_dataset("zloelias/lenta-ru", split = 'test')

In [ ]:
sentences = [s.replace('\xa0', ' ').strip() for s in news_dataset['title']]
sentences[:10]

In [ ]:
sentences = sentences[:1000]

In [ ]:
bs = 8
vectors = []
for i in tdqm(range(0,len(sentences),bs)):
    vectors += vectorize_batch(sentences[i:i+bs],False)

In [ ]:
vectors = [v.numpy() for v in vectors]
vectors = np.vstack(vectors)
vectors.shape

In [ ]:
index = faiss.IndexFlatIP(vectors.shape(1))
index.add(vectors)
index.ntotal

In [ ]:
D, I = index.search(vectorize_single('Киберспорт в России'), 10)
I

In [ ]:
for j, i in enumerate(I[0]):
    print(D[0][j], sentences[i])

Milvus example

In [ ]:
import numpy as np
import time

from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

connections.connect("default", host="localhost", port="19530")

In [ ]:
fields = [
    FieldSchema(name="id", dtype=DataType.VARCHAR, is_primary=True, auto_id=False, max_length=100),
    FieldSchema(name="csv_name", dtype=DataType.VARCHAR, max_length=10),
    FieldSchema(name="sentence", dtype=DataType.VARCHAR, max_length=1024),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=1024)
]

schema= CollectionSchema(fields, "lenta_ru_e5")
news = Collection("lenta_ru", schema, consistency_level="Strong")

In [ ]:
for i in tdqm(range(0, len(sentences), bs)):
    sentences_batch = sentences[i:i+bs]
    vectors_batch = [v.numpy() for v in vectorize_batch(sentences_batch, False)]
    entities = [
        ['bs' + str(i//bs) + '_' + str(j) for j in range(len(sentences_batch))],
        [0.csv] * len(sentences_batch),
        sentences_batch,
        np.array(vectors_batch),
    ]
    insert_result = news.insert(entities)
news.flush()

In [ ]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "COSINE",
    "params": {"nlist": 1024},
}

news.create_index("embeddings", index)
news.load()

In [ ]:
from tqdm import tqdm
search_params = {
    "metric_type": "COSINE",
    "params": {"nprobe": 10},
}

query = 'Киберспорт в России'
query_vec = vectorize_single(query)[0]
result = news.search([query_vec], "embeddings", search_params, limit=10, output_fields=["sentence"])
for hits in result:
    for hit in hits:
        print(hit)


Qdrant examples quickstart with server

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

qdrant_client = QdrantClient("localhost", port=6333, timeout=100)
qdrant_client.recreate_collection(
    collection_name='qdrant_news_lenta',
    vector_config = models.VectorParams(
        size=1024,
        distance=models.Distance.COSINE
    ),
)

Qdrant woking experience example

In [ ]:
from qdrant_client import QdrantClient
qdrant_client.recreate_collection(
    collection_name='qdrant_news_lenta',
    vectors_config = models.VectorParams(
        size=1024,
        distance=models.Distance.COSINE
    ),
)

<ipython-input-38-0a4786459ff1>:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [ ]:
from datasets import load_dataset
dataset = load_dataset('zloelias/lenta-ru', split='train')
dataset

Dataset({
    features: ['title', 'text', 'topic', 'labels'],
    num_rows: 185972
})

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name = 'intfloat/multilingual-e5-large',
    model_kwargs={"device":"cuda"},
    )
def vectorize_batch(sentences):
    return embeddings.embed_documents(sentences)


<ipython-input-7-586646a2f222>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm
import numpy as np
from qdrant_client.http import models

sentences = dataset["title"]
bs = 16

for i in tqdm(range(0, len(sentences)-bs, bs)):
    sentences_batch = sentences[i:i+bs]
    vectors_batch = np.array(vectorize_batch(sentences_batch))
    ids = [j for j in range(i,i + bs)]
    qdrant_client.upsert(
        collection_name='qdrant_news_lenta',
        points=models.Batch(
            payloads=[
                {
                    "sentence": sent,
                    "texts": dataset["text"][i:i+bs],
                    "csv_name": 'lenta-ru.csv',
                }
                for sent in sentences_batch
            ],
            vectors=[v.tolist() for v in vectors_batch],
            ids=ids,
        )
    )

 11%|█         | 1999/18598 [01:52<15:12, 18.20it/s]<ipython-input-21-2ed8b9475433>:13: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 20010 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  qdrant_client.upsert(
100%|█████████▉| 18597/18598 [17:32<00:00, 17.67it/s]


IndexError: list index out of range

No docker qdrant quickstart

In [ ]:
from qdrant_client import QdrantClient

db_path = ""

client = QdrantClient(":memory:")
# or
client = QdrantClient(path=)

In [ ]:
def vectorize_single(sentence):
    res = embeddings.embed_query(sentence)
    res = [r for r in res]
    return res

query = 'Киберспорт в России'
query_vec = vectorize_single(query)

closest_vectors = qdrant_client.query_points(
    collection_name='qdrant_news_lenta',
    query=query_vec,
    limit=10,
    with_vectors=False,
    with_payload=True
)
closest_vectors

QueryResponse(points=[ScoredPoint(id=55221, version=0, score=0.8960727872830734, payload={'sentence': 'Киберспорт снова признали видом спорта в\xa0России', 'csv_name': '0.csv'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=73309, version=0, score=0.8726124312581147, payload={'sentence': 'Сборная России победила на\xa0киберспортивном турнире', 'csv_name': '0.csv'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=23790, version=0, score=0.8653274722590738, payload={'sentence': 'В России появятся менеджеры для киберспорта', 'csv_name': '0.csv'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=15299, version=0, score=0.8652661270515838, payload={'sentence': 'Российских киберспортсменов допустили к\xa0играм на\xa0Xbox 360', 'csv_name': '0.csv'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=177868, version=0, score=0.857303952683776, payload={'sentence': 'В обществе «Спартак» появился киберспортивный клуб', 'csv_name': '0.csv'

langchain vectorize embeddings

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name = 'intfloat/multilingual-e5-large',
    model_kwargs={"device":"cuda"},
    )


<ipython-input-2-8fcfed4a8be5>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
import numpy as np
query = 'Киберспорт в России'
vec = np.array(embeddings.embed_query(f"query: {query}"))
vec

array([-0.00479927, -0.00663975, -0.03167529, ...,  0.00573857,
       -0.01711249,  0.00577609])

Langchain and qdrant

In [ ]:
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(":memory:")
doc_store = Qdrant(qdrant_client,
                   collection_name = "qdrant_news_lenta",
                   embeddings = embeddings,
                   content_payload_key = "sentence")

<ipython-input-4-6f24a01c90a0>:5: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  doc_store = Qdrant(qdrant_client,


In [ ]:
query = "Цены на нефть"

In [ ]:
from qdrant_client.http import models
qdrant_client.recreate_collection(
    collection_name='qdrant_news_lenta',
    vectors_config = models.VectorParams(
        size=1024,
        distance=models.Distance.COSINE
    ),
)

<ipython-input-10-8e12fd83c07e>:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [ ]:
retriever = doc_store.as_retriever(search_type='similarity',search_kwargs={'k' : 10})
retriever.get_relevant_documents("query: " + query)

[]

In [ ]:
retriever = doc_store.as_retriever(search_type='mmr',search_kwargs={'k' : 10})
retriever.get_relevant_documents("query: " + query)

ResponseHandlingException: [Errno 99] Cannot assign requested address

In [ ]:
from langchain_core.documents import Document
from langchain_community.retrievers import BM25Retriever
import pymorphy2
parser = pymorphy2.MorphAnalyzer()

def preprocess(text):
    words = text.lower().split()
    words = [parser.parse(w)[0].normal_form for w in words]
    return words

print(preprocess('Цены на нефть снижаются'))
docs = [Document(page_content=point.payload['sentence'],metadata={'id': point.id})
        for point in qdrant_client.scroll(
                collection_name="qdrant_news_lenta", limit=qdrant_client.get_collection('qdrant_news_lenta').vectors_count
        )[0]]

bm25_retriever = BM25Retriever.from_documents(docs, preprocess_func=preprocess, k=10)

AttributeError: module 'inspect' has no attribute 'getargspec'

In [ ]:
bm25_retriever.get_relevant_documents(query)

Langchain+llm

In [ ]:
from transformers import AutoTokenizer, AutoModelForCasualLM
import torch
model_name = 'openchat/openchat-3.5-1210'
model = AutoModelForCasualLM.from_pretrained(model_name, torch_dtype=torch.float16, attn_implementation='flash_attention_2', device_map='auto')
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from transformers import pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

pipeline_kwargs={"max_new_tokens": 128, "do_sample": True, "temperature": 0.2}
pipe = pipeline("text-generation",model=model, tokenizer=tokenizer, **pipeline_kwargs)
hugging_face_pipeline = HuggingFacePipeline(pipeline=pipe)


In [ ]:
query = "Как дела с киберспортом в РФ?"
relevant_docs = retriever.get_relevant_documents('query: ' + query)
docs_text = '\n\n'.join([doc.page_content for doc in relevant_docs])
chat = [
    {"role": "user", "content": f"Релевантная информация: {docs_text}\n\nВопрос: {query}"},
]
relevant_docs

In [ ]:
def apply_chat(messages):
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt
chain = apply_chat | hugging_face_pipeline
res= chain.invoke(chat)
print(res[res.rfind('GPT4 Correct Assistant:'):])